In [ ]:
pip install clarifai

In [1]:
import os

In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/workspaces/Module-Data-Cleanser/Customer Churn.csv')

# Function to create the row_contents string for each row
def create_row_contents(row):
    contents = []
    for col in df.columns:
        contents.append(f"{col.replace(' ', '_')} is {row[col]}")
    return ', '.join(contents)

# Add row_contents column to the DataFrame
df['row_contents'] = df.apply(create_row_contents, axis=1)

# Print the DataFrame with the added row_contents column
df


,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn,row_contents
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,0,"Call__Failure is 8.0, Complains is 0.0, Subscr..."
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035,0,"Call__Failure is 0.0, Complains is 0.0, Subscr..."
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,0,"Call__Failure is 10.0, Complains is 0.0, Subsc..."
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,0,"Call__Failure is 10.0, Complains is 0.0, Subsc..."
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0,"Call__Failure is 3.0, Complains is 0.0, Subscr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,21,0,19,2,6697,147,92,44,2,2,1,25,721.980,0,"Call__Failure is 21.0, Complains is 0.0, Subsc..."
3146,17,0,17,1,9237,177,80,42,5,1,1,55,261.210,0,"Call__Failure is 17.0, Complains is 0.0, Subsc..."
3147,13,0,18,4,3157,51,38,21,3,1,1,30,280.320,0,"Call__Failure is 13.0, Complains is 0.0, Subsc..."
3148,7,0,11,2,4695,46,222,12,3,1,1,30,1077.640,0,"Call__Failure is 7.0, Complains is 0.0, Subscr..."


In [18]:
listofcontents = df["row_contents"][0:300].tolist()


In [21]:
import os
import uuid
import pandas as pd
from clarifai.client.model import Model
from clarifai.client.input import Inputs

def get_embeddings_for_row_content(model_name: str, _PAT, row_content):
    try:
        model_obj = Model(model_name, pat =_PAT)
        input_obj = Inputs(pat = _PAT)
        batch_size = 32
        embeddings = []
        print(f" len of row content {len(row_content)}")

        for i in range(0, len(row_content), batch_size):
            batch = row_content[i : i + batch_size]
            print(f"computing embeddings {i} to {i + batch_size}")
            input_batch = [
                input_obj.get_text_input(input_id=(uuid.uuid4().hex), raw_text=inp)
                for id, inp in enumerate(batch)
            ]
            predict_response = model_obj.predict(input_batch)
            print(f"model prediction response: {predict_response}")
            embeddings.extend(
                [
                    list(output.data.embeddings[0].vector)
                    for output in predict_response.outputs
                ]
            )
        return embeddings

    
    except Exception as e:
        print(f"Embedding predictions failed due to {e}")
        return None

In [ ]:
embeddings=get_embeddings_for_row_content("https://clarifai.com/openai/embed/models/text-embedding-3-small", "", listofcontents)

In [23]:
len(embeddings)

300